In [7]:
#### What happens when ONLY mode mismatch the FC into the IFO?

### we need to make sure we lock in the Sqz Mode

In [8]:
%matplotlib inline
import pykat
import pylab as pl
import numpy as np

### My functions
from kat_commands import *
from cav_commands import *
from load_kat_data import *
from MMT import modematch

import get_WS
import trace_mode

In [12]:
katfile = "aLIGO_OnlySqz_OMC.kat"
maxtem = 0

In [38]:
#load the original kat file (ie Perfect Modematching) as a reference
basekat = pykat.finesse.kat() 
basekat.loadKatFile(katfile)
basekat.parseCommands(darm_commands)
basekat.parseCommands("maxtem "+str(maxtem))
basekat.parseCommands("""
bp q_SRMHR_x x q nSRMHRa
bp q_SRMHR_y y q nSRMHRa

bp q_sqz_x x q nOPO_HRIC_in
bp q_sqz_y y q nOPO_HRIC_in

bp q_OMC_x x q nOMC_HRIC_in
bp q_OMC_y y q nOMC_HRIC_in
""")
basekat.undumpNodes() #undump the vacuum nodes

Parsing `dbs OPP nOPb nOPc nOPd nOPz` into pykat object not implemented yet, added as extra line.


In [28]:
#Set the mode of the Sqz
kat=basekat.deepcopy() #copy kat code
kat.verbose = False
kat.parseCommands("""
gauss gaussOPOx OPO_HRIC nOPO_HRIC_in 187.9u .055 200.2u 0.055
""")
out = kat.run()

In [29]:
#Get the mode of the OMC
kat1 = basekat.deepcopy()
kat1.parseCommands("""
cav cavOMC OMC_HROC nOMC_HROC_refl OMC_HROC nOMC_HROC_in
""")
outOMC = kat1.run()

--------------------------------------------------------------
Running kat - Started at 2018-02-23 16:15:35.029510


 63% | ETA:  0:00:00 | Calculating                                             


Finished in 1.59524 seconds


100% | ETA:  0:00:00 | Saving data                                             

In [32]:
#Get the mode of the SRC
kat2 = basekat.deepcopy()
kat2.parseCommands("""
cav cavSRX SRMHR nSRMHRa ITMXHR nITMXs2
cav cavSRY SRMHR nSRMHRa ITMYHR nITMYs2
""")
outSRC = kat2.run()

--------------------------------------------------------------
Running kat - Started at 2018-02-23 16:17:03.752502


 63% | ETA:  0:00:00 | Calculating                                             


Finished in 1.48194 seconds


100% | ETA:  0:00:00 | Saving data                                             

In [23]:
q_sqz_x = out["q_sqz_x"][0]
q_sqz_y = out["q_sqz_y"][0]

q_sqz_x_obj = pykat.BeamParam(q=q_sqz_x)
q_sqz_y_obj = pykat.BeamParam(q=q_sqz_y)

In [ ]:
kat.nodes.getComponentsBetween('nOMC_HROC_refl','nOPO_HRIC_in')

In [ ]:
pykat.BeamParam.overlap(-np.conjugate(out_mismatch_sqz['q_sqz_x'][0]),out_perfect_mm['q_sqz_x'][0])

In [ ]:
##### Testing to trace the modes
#### Turn off all the cav commands except SRCY to extract the eigenmode at the BS
[srcx,srcy] = trace_mode.from_cav_to_BS(kat0,'cavSRY')

#### Turn off all the cav commands except XARM to extract the eigenmode at the BS
[ARMx,ARMy] = trace_mode.from_cav_to_BS(kat0,'cavXARM')

#### Turn off all the cav commands except OMC to extract the eigenmode at the BS
[OMCx,OMCy] = trace_mode.from_cav_to_BS(kat0,'cavOMC')

#### Turn off all the cav commands except FC to extract the eigenmode at the BS
[FCx,FCy] = trace_mode.from_cav_to_BS(kat0,'cavFC')